In [1]:
import cv2 as cv
import numpy as np

In [ ]:
from collections import deque
import cv2
import numpy as np

def setValue(x):
    print('')
cap=cv2.VideoCapture(0)

window=cv2.namedWindow('window')
cv2.createTrackbar('upper hue','window',153,180,setValue)
cv2.createTrackbar('upper saturation','window',255,255,setValue)
cv2.createTrackbar('upper value','window',255,255,setValue)
cv2.createTrackbar('lower hue','window',64,180,setValue)
cv2.createTrackbar('lower saturation','window',171,255,setValue)
cv2.createTrackbar('lower value','window',78,255,setValue)

bpoints=[deque(maxlen=1024)]
gpoints=[deque(maxlen=1024)]
rpoints=[deque(maxlen=1024)]
ypoints=[deque(maxlen=1024)]

blue_index=0
green_index=0
red_index=0
yellow_index=0

kernel=np.ones((3,3),np.uint8)

colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (0, 255,255)]
colorIndex = 0

#Creating Paint Window

paintWin = np.zeros([480,640,3]) + 255
paintWin = cv2.rectangle(paintWin, (30,1), (120,65), (0,0,0) , 0 )
paintWin = cv2.rectangle(paintWin, (160,1) , (255,65) , colors[0], -1)
paintWin = cv2.rectangle(paintWin, (280,1) , (370,65) , colors[1], -1)
paintWin = cv2.rectangle(paintWin, (400,1) , (485,65) , colors[2], -1)
paintWin = cv2.rectangle(paintWin, (520,1) , (600,65) , colors[3], -1)

cv2.putText(paintWin, "CLEAR",(58,40), cv2.FONT_HERSHEY_SIMPLEX, 0.5 , (0, 0, 0), 1, cv2.LINE_AA)
cv2.putText(paintWin, "BLUE",(187,40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
cv2.putText(paintWin, "GREEN",(295,40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
cv2.putText(paintWin, "RED",(415,40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
cv2.putText(paintWin, "YELLOW",(530,40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

cap=cv2.VideoCapture(0)
while True:
    ret,frame=cap.read()
    #flip to see the same side
    frame=cv2.flip(frame,1)
    hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    
    u_hue=cv2.getTrackbarPos('upper hue','window')
    u_saturation=cv2.getTrackbarPos('upper saturation','window') 
    u_value=cv2.getTrackbarPos('upper value','window')      
    l_hue=cv2.getTrackbarPos('lower hue','window')         
    l_saturation=cv2.getTrackbarPos('lower saturation','window')         
    l_value=cv2.getTrackbarPos('lower value','window')   

    upper_hsv=np.array([u_hue,u_saturation,u_value])
    lower_hsv=np.array([l_hue,l_saturation,l_value])
    
    # Shows color Options in Video Outputed by Webcam 

    frame = cv2.rectangle(frame, (30,1), (120,65), (122,122,122) , -1 )
    frame = cv2.rectangle(frame, (160,1) , (255,65) , colors[0], -1)
    frame = cv2.rectangle(frame, (280,1) , (370,65) , colors[1], -1)
    frame = cv2.rectangle(frame, (400,1) , (485,65) , colors[2], -1)
    frame = cv2.rectangle(frame, (520,1) , (600,65) , colors[3], -1)

    cv2.putText(frame, "CLEAR ALL",(58,40), cv2.FONT_HERSHEY_SIMPLEX, 0.5 , (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(frame, "BLUE",(187,40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
    cv2.putText(frame, "GREEN",(295,40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
    cv2.putText(frame, "RED",(415,40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
    cv2.putText(frame, "YELLOW",(530,40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
    
     # Making A mask and dilating and eroding to remove impurities.  

    mask = cv2.inRange(hsv,lower_hsv,upper_hsv)
    mask = cv2.erode(mask,kernel,iterations=1)
    mask = cv2.morphologyEx(mask,cv2.MORPH_OPEN,kernel)
    mask = cv2.dilate(mask,kernel,iterations=1)
    
    cnt,_ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    center = None
    
    if len(cnt) > 0:
        # Sorting the contours on basis of their length
        cnt = sorted(cnt,key = cv2.contourArea, reverse=True)[0]
        # Finding the minimum area of contour
        ((x,y),radius) = cv2.minEnclosingCircle(cnt)
        # Drawing the circle around the contour
        cv2.circle(frame, (int(x),int(y)), int(radius), (0,255,255), 2)

        # Finding the area and centroid of the contour
        M = cv2.moments(cnt)
        center = (int(M['m10'] / M['m00']) , int(M['m01'] / M['m00']))


        # Drawing line on the screen
        if center[1] <= 65:
            if 30 <= center[0] <= 120: #Clear Button

                bpoints = [deque(maxlen=512)]
                gpoints = [deque(maxlen=512)]
                rpoints = [deque(maxlen=512)]
                bpoints = [deque(maxlen=512)]

                blue_index = 0
                green_index = 0 
                red_index = 0
                yellow_index = 0

                paintWin[67:,:,:] = 255
            elif 160 <= center[0] <= 255:
                    colorIndex = 0 # Blue
            elif 280 <= center[0] <= 370:
                    colorIndex = 1 # Green
            elif 400 <= center[0] <= 485:
                    colorIndex = 2 # Red
            elif 520 <= center[0] <= 600:
                    colorIndex = 3 # 
        else :
            if colorIndex == 0:
                bpoints[blue_index].appendleft(center)
            elif colorIndex == 1:
                gpoints[green_index].appendleft(center)
            elif colorIndex == 2:
                rpoints[red_index].appendleft(center)
            elif colorIndex == 3:
                bpoints[yellow_index].appendleft(center)

    else:
        bpoints.append(deque(maxlen=512))
        blue_index += 1
        gpoints.append(deque(maxlen=512))
        green_index += 1
        rpoints.append(deque(maxlen=512))
        red_index += 1
        bpoints.append(deque(maxlen=512))
        yellow_index += 1

        # Draw lines of all the colors on the canvas and frame 
        points = [bpoints, gpoints, rpoints, bpoints]
        for i in range(len(points)):
            for j in range(len(points[i])):
                for k in range(1, len(points[i][j])):
                    if points[i][j][k - 1] is None or points[i][j][k] is None:
                        continue
                    cv2.line(frame, points[i][j][k - 1], points[i][j][k], colors[i], 2)
                    cv2.line(paintWin, points[i][j][k - 1], points[i][j][k], colors[i], 2)

    cv2.imshow("Air Canvas",frame)
    cv2.imshow("Mask",mask)
    cv2.imshow("Paint", paintWin)
    if cv2.waitKey(50) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()